<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/ugcqz6ohbvff804xp84y4kqnvvk3bq1g.png" width = 300, align = "center"></a>

<h1 align=center><font size = 5>Lab: Working with a real world data-set using SQL and Python</font></h1>

# Introduction

This notebook shows how to work with a real world dataset using SQL and Python. In this lab you will:
1. Understand the dataset for Chicago Public School level performance 
1. Store the dataset in an Db2 database on IBM Cloud instance
1. Retrieve metadata about tables and columns and query data from mixed case columns
1. Solve example problems to practice your SQL skills including using built-in database functions

## Chicago Public Schools - Progress Report Cards (2011-2012) 

The city of Chicago released a dataset showing all school level performance data used to create School Report Cards for the 2011-2012 school year. The dataset is available from the Chicago Data Portal: https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t

This dataset includes a large number of metrics. Start by familiarizing yourself with the types of metrics in the database: https://data.cityofchicago.org/api/assets/AAD41A13-BE8A-4E67-B1F5-86E711E09D5F?download=true

__NOTE__: Do not download the dataset directly from City of Chicago portal. Instead download a more database friendly version from the link below.
Now download a static copy of this database and review some of its contents:
https://ibm.box.com/shared/static/f9gjvj1gjmxxzycdhplzt01qtz0s7ew7.csv



### Store the dataset in a Table
In many cases the dataset to be analyzed is available as a .CSV (comma separated values) file, perhaps on the internet. To analyze the data using SQL, it first needs to be stored in the database.

While it is easier to read the dataset into a Pandas dataframe and then PERSIST it into the database as we saw in the previous lab, it results in mapping to default datatypes which may not be optimal for SQL querying. For example a long textual field may map to a CLOB instead of a VARCHAR. 

Therefore, __it is highly recommended to manually load the table using the database console LOAD tool, as indicated in Week 2 Lab 1 Part II__. The only difference with that lab is that in Step 5 of the instructions you will need to click on create "(+) New Table" and specify the name of the table you want to create and then click "Next". 

##### Now open the Db2 console, open the LOAD tool, Select / Drag the .CSV file for the CHICAGO PUBLIC SCHOOLS dataset and load the dataset into a new table called __SCHOOLS__.

<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/uc4xjh1uxcc78ks1i18v668simioz4es.jpg"></a>

### Connect to the database
Let us now load the ipython-sql  extension and establish a connection with the database

In [2]:
%load_ext sql

In [3]:
# Enter the connection string for your Db2 on Cloud database instance below
# %sql ibm_db_sa://my-username:my-password@my-hostname:my-port/my-db-name
%sql ibm_db_sa://mxz21962:clc1d32ktmpt%2B2f3@dashdb-txn-sbox-yp-dal09-14.services.dal.bluemix.net:50000/BLUDB

'Connected: mxz21962@BLUDB'

### Query the database system catalog to retrieve table metadata

##### You can verify that the table creation was successful by retrieving the list of all tables in your schema and checking whether the SCHOOLS table was created

In [4]:
# type in your query to retrieve list of all tables in the database for your db2 schema (username)
%sql select TABSCHEMA, TABNAME, CREATE_TIME from SYSCAT.TABLES where TABSCHEMA='MXZ21962'

 * ibm_db_sa://mxz21962:***@dashdb-txn-sbox-yp-dal09-14.services.dal.bluemix.net:50000/BLUDB
Done.


tabschema,tabname,create_time
MXZ21962,INSTRUCTOR,2020-08-31 02:17:25.244137
MXZ21962,EMPLOYEES,2020-09-01 02:14:17.426588
MXZ21962,JOB_HISTORY,2020-09-01 02:14:17.898970
MXZ21962,JOBS,2020-09-01 02:14:18.061145
MXZ21962,DEPARTMENTS,2020-09-01 02:14:18.223147
MXZ21962,LOCATIONS,2020-09-01 02:14:18.389014
MXZ21962,PETSALE,2020-09-01 04:27:45.408517
MXZ21962,INTERNATIONAL_STUDENT_TEST_SCORES,2020-09-02 04:57:17.307794
MXZ21962,CHICAGO_SOCIOECONOMIC_DATA,2020-09-02 05:08:32.684273
MXZ21962,SCHOOLS,2020-09-03 14:22:21.019983


In [5]:
%sql select TABSCHEMA, TABNAME, CREATE_TIME from SYSCAT.TABLES \
      where TABSCHEMA not in ('SYSIBM', 'SYSCAT', 'SYSSTAT', 'SYSIBMADM', 'SYSTOOLS', 'SYSPUBLIC')

 * ibm_db_sa://mxz21962:***@dashdb-txn-sbox-yp-dal09-14.services.dal.bluemix.net:50000/BLUDB
Done.


tabschema,tabname,create_time
MXZ21962,INSTRUCTOR,2020-08-31 02:17:25.244137
MXZ21962,EMPLOYEES,2020-09-01 02:14:17.426588
MXZ21962,JOB_HISTORY,2020-09-01 02:14:17.898970
MXZ21962,JOBS,2020-09-01 02:14:18.061145
MXZ21962,DEPARTMENTS,2020-09-01 02:14:18.223147
MXZ21962,LOCATIONS,2020-09-01 02:14:18.389014
MXZ21962,PETSALE,2020-09-01 04:27:45.408517
MXZ21962,INTERNATIONAL_STUDENT_TEST_SCORES,2020-09-02 04:57:17.307794
MXZ21962,CHICAGO_SOCIOECONOMIC_DATA,2020-09-02 05:08:32.684273
MXZ21962,SCHOOLS,2020-09-03 14:22:21.019983


In [6]:
%sql select * from SYSCAT.TABLES where TABNAME = 'SCHOOLS'

 * ibm_db_sa://mxz21962:***@dashdb-txn-sbox-yp-dal09-14.services.dal.bluemix.net:50000/BLUDB
Done.


tabschema,tabname,owner,ownertype,TYPE,status,base_tabschema,base_tabname,rowtypeschema,rowtypename,create_time,alter_time,invalidate_time,stats_time,colcount,tableid,tbspaceid,card,npages,mpages,fpages,npartitions,nfiles,tablesize,overflow,tbspace,index_tbspace,long_tbspace,parents,children,selfrefs,keycolumns,keyindexid,keyunique,checkcount,datacapture,const_checked,pmap_id,partition_mode,log_attribute,pctfree,append_mode,REFRESH,refresh_time,LOCKSIZE,VOLATILE,row_format,property,statistics_profile,compression,rowcompmode,access_mode,clustered,active_blocks,droprule,maxfreespacesearch,avgcompressedrowsize,avgrowcompressionratio,avgrowsize,pctrowscompressed,logindexbuild,codepage,collationschema,collationname,collationschema_orderby,collationname_orderby,encoding_scheme,pctpagessaved,last_regen_time,secpolicyid,protectiongranularity,auditpolicyid,auditpolicyname,auditexceptionenabled,definer,oncommit,logged,onrollback,lastused,control,temporaltype,tableorg,extended_row_size,pctextendedrows,remarks
MXZ21962,SCHOOLS,MXZ21962,U,T,N,None,None,None,None,2020-09-03 14:22:21.019983,2020-09-03 14:22:21.019983,2020-09-03 14:22:21.019983,2020-09-03 14:29:52.749055,78,13,1909,566,15,0,16,-1,-1,-1,0,mxz21962space1,None,None,0,0,0,0,0,0,0,N,YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY,1,,0,-1,N,,None,R,,N,,None,N,,F,None,0,N,999,0,0.0,844,0.0,None,1208,SYSIBM,IDENTITY,SYSIBM,IDENTITY,,0,2020-09-03 14:22:21.019983,0,,None,None,N,MXZ21962,,,,2020-09-03,,N,R,N,-1.0,None


Double-click __here__ for a hint

<!--
In Db2 the system catalog table called SYSCAT.TABLES contains the table metadata
-->

Double-click __here__ for the solution.

<!-- Solution:

%sql select TABSCHEMA, TABNAME, CREATE_TIME from SYSCAT.TABLES where TABSCHEMA='YOUR-DB2-USERNAME'

or, you can retrieve list of all tables where the schema name is not one of the system created ones:

%sql select TABSCHEMA, TABNAME, CREATE_TIME from SYSCAT.TABLES \
      where TABSCHEMA not in ('SYSIBM', 'SYSCAT', 'SYSSTAT', 'SYSIBMADM', 'SYSTOOLS', 'SYSPUBLIC')
      
or, just query for a specifc table that you want to verify exists in the database
%sql select * from SYSCAT.TABLES where TABNAME = 'SCHOOLS'

-->

### Query the database system catalog to retrieve column metadata

##### The SCHOOLS table contains a large number of columns. How many columns does this table have?

In [7]:
# type in your query to retrieve the number of columns in the SCHOOLS table
%sql select * from sysibm.syscolumns where tbname = 'SCHOOLS'


 * ibm_db_sa://mxz21962:***@dashdb-txn-sbox-yp-dal09-14.services.dal.bluemix.net:50000/BLUDB
Done.


name,tbname,tbcreator,remarks,coltype,NULLS,codepage,dbcscodepg,length,scale,colno,colcard,high2key,low2key,avgcollen,keyseq,typename,typeschema,DEFAULT,longlength,logged,compact,nquantiles,nmostfreq,composite_codepage,partkeyseq,source_tabschema,source_tabname,hidden,GENERATED,inline_length,numnulls,avgcollenchar,sub_count,sub_delim_length,IDENTITY,compress,avgdistinctperpage,pagevarianceratio,implicitvalue,seclabelid,collationid,pctinlined,pctencoded,avgencodedcollen
School_ID,SCHOOLS,MXZ21962,None,INTEGER,Y,0,0,4,0,0,566,610543,609674,5,None,INTEGER,SYSIBM,None,4,,,20,10,0,0,None,None,,,0,0,-1,-1,-1,N,O,None,-1.0,None,0,None,-1,-1,-1.0
NAME_OF_SCHOOL,SCHOOLS,MXZ21962,None,VARCHAR,Y,1208,1200,65,0,1,566,'Woodlawn Community Elementary Sch','Abraham Lincoln Elementary School',41,None,VARCHAR,SYSIBM,None,65,,,20,10,1208,0,None,None,,,0,0,36,-1,-1,N,O,None,-1.0,None,0,b'\x00\x00\x00\x02\xff\x00\xff\xff',-1,-1,-1.0
"Elementary, Middle, or High School",SCHOOLS,MXZ21962,None,VARCHAR,Y,1208,1200,2,0,2,3,'MS','ES',7,None,VARCHAR,SYSIBM,None,2,,,20,10,1208,0,None,None,,,0,0,2,-1,-1,N,O,None,-1.0,None,0,b'\x00\x00\x00\x02\xff\x00\xff\xff',-1,-1,-1.0
Street_Address,SCHOOLS,MXZ21962,None,VARCHAR,Y,1208,1200,30,0,3,566,'9912 S Avenue H','10015 S Leavitt St',22,None,VARCHAR,SYSIBM,None,30,,,20,10,1208,0,None,None,,,0,0,17,-1,-1,N,O,None,-1.0,None,0,b'\x00\x00\x00\x02\xff\x00\xff\xff',-1,-1,-1.0
City,SCHOOLS,MXZ21962,None,VARCHAR,Y,1208,1200,7,0,4,1,'Chicago','Chicago',12,None,VARCHAR,SYSIBM,None,7,,,20,10,1208,0,None,None,,,0,0,7,-1,-1,N,O,None,-1.0,None,0,b'\x00\x00\x00\x02\xff\x00\xff\xff',-1,-1,-1.0
State,SCHOOLS,MXZ21962,None,VARCHAR,Y,1208,1200,2,0,5,1,'IL','IL',7,None,VARCHAR,SYSIBM,None,2,,,20,10,1208,0,None,None,,,0,0,2,-1,-1,N,O,None,-1.0,None,0,b'\x00\x00\x00\x02\xff\x00\xff\xff',-1,-1,-1.0
ZIP_Code,SCHOOLS,MXZ21962,None,INTEGER,Y,0,0,4,0,6,51,60707,60607,5,None,INTEGER,SYSIBM,None,4,,,20,10,0,0,None,None,,,0,0,-1,-1,-1,N,O,None,-1.0,None,0,None,-1,-1,-1.0
Phone_Number,SCHOOLS,MXZ21962,None,VARCHAR,Y,1208,1200,14,0,7,566,'(773) 535-9930','(773) 534-0146',19,None,VARCHAR,SYSIBM,None,14,,,20,10,1208,0,None,None,,,0,0,14,-1,-1,N,O,None,-1.0,None,0,b'\x00\x00\x00\x02\xff\x00\xff\xff',-1,-1,-1.0
Link,SCHOOLS,MXZ21962,None,VARCHAR,Y,1208,1200,78,0,8,2,'','http://schoolreports.cps.edu/Scho',83,None,VARCHAR,SYSIBM,None,78,,,20,10,1208,0,None,None,,,0,1,78,-1,-1,N,O,None,-1.0,None,0,b'\x00\x00\x00\x02\xff\x00\xff\xff',-1,-1,-1.0
Network_Manager,SCHOOLS,MXZ21962,None,VARCHAR,Y,1208,1200,40,0,9,20,'Southwest Side High School Networ','Austin-North Lawndale Elementary ',35,None,VARCHAR,SYSIBM,None,40,,,20,10,1208,0,None,None,,,0,0,30,-1,-1,N,O,None,-1.0,None,0,b'\x00\x00\x00\x02\xff\x00\xff\xff',-1,-1,-1.0


In [8]:
%sql select count(name) from sysibm.syscolumns where tbname = 'SCHOOLS'

 * ibm_db_sa://mxz21962:***@dashdb-txn-sbox-yp-dal09-14.services.dal.bluemix.net:50000/BLUDB
Done.


1
78


In [9]:
%sql select COLNAME, TYPENAME, LENGTH from SYSCAT.COLUMNS where TABNAME = 'SCHOOLS'

 * ibm_db_sa://mxz21962:***@dashdb-txn-sbox-yp-dal09-14.services.dal.bluemix.net:50000/BLUDB
Done.


colname,typename,length
School_ID,INTEGER,4
NAME_OF_SCHOOL,VARCHAR,65
"Elementary, Middle, or High School",VARCHAR,2
Street_Address,VARCHAR,30
City,VARCHAR,7
State,VARCHAR,2
ZIP_Code,INTEGER,4
Phone_Number,VARCHAR,14
Link,VARCHAR,78
Network_Manager,VARCHAR,40


In [10]:
%sql select count(*) from SYSCAT.COLUMNS where TABNAME = 'SCHOOLS'

 * ibm_db_sa://mxz21962:***@dashdb-txn-sbox-yp-dal09-14.services.dal.bluemix.net:50000/BLUDB
Done.


1
78


Double-click __here__ for a hint

<!--
In Db2 the system catalog table called SYSCAT.COLUMNS contains the column metadata
-->

Double-click __here__ for the solution.

<!-- Solution:

%sql select count(*) from SYSCAT.COLUMNS where TABNAME = 'SCHOOLS'

-->

Now retrieve the the list of columns in SCHOOLS table and their column type (datatype) and length.

In [11]:
# type in your query to retrieve all column names in the SCHOOLS table along with their datatypes and length
%sql select * from SYSCAT.COLUMNS where TABNAME = 'SCHOOLS' limit 1

 * ibm_db_sa://mxz21962:***@dashdb-txn-sbox-yp-dal09-14.services.dal.bluemix.net:50000/BLUDB
Done.


tabschema,tabname,colname,colno,typeschema,typename,length,scale,typestringunits,stringunitslength,DEFAULT,NULLS,codepage,collationschema,collationname,logged,compact,colcard,high2key,low2key,avgcollen,keyseq,partkeyseq,nquantiles,nmostfreq,numnulls,target_typeschema,target_typename,scope_tabschema,scope_tabname,source_tabschema,source_tabname,dl_features,special_props,hidden,inline_length,pctinlined,IDENTITY,rowchangetimestamp,GENERATED,text,compress,avgdistinctperpage,pagevarianceratio,sub_count,sub_delim_length,avgcollenchar,implicitvalue,seclabelname,rowbegin,rowend,transactionstartid,pctencoded,avgencodedcollen,qualifier,func_path,randdistkey,remarks
MXZ21962,SCHOOLS,School_ID,0,SYSIBM,INTEGER,4,0,None,None,None,Y,0,None,None,,,566,610543,609674,5,None,0,20,10,0,None,None,None,None,None,None,None,None,,0,-1,N,N,,None,O,None,-1.0,-1,-1,-1,None,None,N,N,N,-1,-1.0,None,None,N,None


In [12]:
%sql select * from sysibm.syscolumns where tbname = 'SCHOOLS' limit 1

 * ibm_db_sa://mxz21962:***@dashdb-txn-sbox-yp-dal09-14.services.dal.bluemix.net:50000/BLUDB
Done.


name,tbname,tbcreator,remarks,coltype,NULLS,codepage,dbcscodepg,length,scale,colno,colcard,high2key,low2key,avgcollen,keyseq,typename,typeschema,DEFAULT,longlength,logged,compact,nquantiles,nmostfreq,composite_codepage,partkeyseq,source_tabschema,source_tabname,hidden,GENERATED,inline_length,numnulls,avgcollenchar,sub_count,sub_delim_length,IDENTITY,compress,avgdistinctperpage,pagevarianceratio,implicitvalue,seclabelid,collationid,pctinlined,pctencoded,avgencodedcollen
School_ID,SCHOOLS,MXZ21962,None,INTEGER,Y,0,0,4,0,0,566,610543,609674,5,None,INTEGER,SYSIBM,None,4,,,20,10,0,0,None,None,,,0,0,-1,-1,-1,N,O,None,-1.0,None,0,None,-1,-1,-1.0


Double-click __here__ for the solution.

<!-- Solution:

%sql select COLNAME, TYPENAME, LENGTH from SYSCAT.COLUMNS where TABNAME = 'SCHOOLS'

or

%sql select distinct(NAME), COLTYPE, LENGTH from SYSIBM.SYSCOLUMNS where TBNAME = 'SCHOOLS'

-->

### Questions
1. Is the column name for the "SCHOOL ID" attribute in upper or mixed case?
1. What is the name of "Community Area Name" column in your table? Does it have spaces?
1. Are there any columns in whose names the spaces and paranthesis (round brackets) have been replaced by the underscore character "_"?

## Problems

### Problem 1

##### How many Elementary Schools are in the dataset?

In [32]:
%sql select count (*) from schools where "Elementary, Middle, or High School" = 'ES'

 * ibm_db_sa://mxz21962:***@dashdb-txn-sbox-yp-dal09-14.services.dal.bluemix.net:50000/BLUDB
Done.


1
462


Double-click __here__ for a hint

<!--
Which column specifies the school type e.g. 'ES', 'MS', 'HS'?
-->

Double-click __here__ for another hint

<!--
Does the column name have mixed case, spaces or other special characters?
If so, ensure you use double quotes around the "Name of the Column"
-->

Double-click __here__ for the solution.

<!-- Solution:

%sql select count(*) from SCHOOLS where "Elementary, Middle, or High School" = 'ES'

Correct answer: 462

-->

### Problem 2

##### What is the highest Safety Score?

In [41]:
%sql select max(safety_score) MAX_SAFETY_SCORE from schools

 * ibm_db_sa://mxz21962:***@dashdb-txn-sbox-yp-dal09-14.services.dal.bluemix.net:50000/BLUDB
Done.


max_safety_score
99


Double-click __here__ for a hint

<!--
Use the MAX() function
-->

Double-click __here__ for the solution.

<!-- Hint:

%sql select MAX(Safety_Score) AS MAX_SAFETY_SCORE from SCHOOLS

Correct answer: 99
-->


### Problem 3

##### Which schools have highest Safety Score?

In [43]:
%sql select Name_of_School, Safety_Score from SCHOOLS where Safety_Score = (select MAX(Safety_Score) from SCHOOLS )


 * ibm_db_sa://mxz21962:***@dashdb-txn-sbox-yp-dal09-14.services.dal.bluemix.net:50000/BLUDB
Done.


name_of_school,safety_score
Abraham Lincoln Elementary School,99
Alexander Graham Bell Elementary School,99
Annie Keller Elementary Gifted Magnet School,99
Augustus H Burley Elementary School,99
Edgar Allan Poe Elementary Classical School,99
Edgebrook Elementary School,99
Ellen Mitchell Elementary School,99
James E McDade Elementary Classical School,99
James G Blaine Elementary School,99
LaSalle Elementary Language Academy,99


Double-click __here__ for the solution.

<!-- Solution:
In the previous problem we found out that the highest Safety Score is 99, so we can use that as an input in the where clause:

%sql select Name_of_School, Safety_Score from SCHOOLS where Safety_Score = 99

or, a better way:

%sql select Name_of_School, Safety_Score from SCHOOLS where \
  Safety_Score= (select MAX(Safety_Score) from SCHOOLS)


Correct answer: several schools with with Safety Score of 99.
-->


### Problem 4

##### What are the top 10 schools with the highest "Average Student Attendance"?


In [48]:
%sql select * from SCHOOLS where Safety_Score = (select MAX(Safety_Score) from SCHOOLS) limit 1

 * ibm_db_sa://mxz21962:***@dashdb-txn-sbox-yp-dal09-14.services.dal.bluemix.net:50000/BLUDB
Done.


School_ID,name_of_school,"Elementary, Middle, or High School",Street_Address,City,State,ZIP_Code,Phone_Number,Link,Network_Manager,Collaborative_Name,Adequate_Yearly_Progress_Made_,Track_Schedule,CPS_Performance_Policy_Status,CPS_Performance_Policy_Level,healthy_school_certified,Safety_Icon,safety_score,Family_Involvement_Icon,Family_Involvement_Score,Environment_Icon,Environment_Score,Instruction_Icon,Instruction_Score,Leaders_Icon,Leaders_Score,Teachers_Icon,Teachers_Score,Parent_Engagement_Icon,Parent_Engagement_Score,Parent_Environment_Icon,Parent_Environment_Score,average_student_attendance,Rate_of_Misconducts__per_100_students_,Average_Teacher_Attendance,Individualized_Education_Program_Compliance_Rate,Pk_2_Literacy__,Pk_2_Math__,Gr3_5_Grade_Level_Math__,Gr3_5_Grade_Level_Read__,Gr3_5_Keep_Pace_Read__,Gr3_5_Keep_Pace_Math__,Gr6_8_Grade_Level_Math__,Gr6_8_Grade_Level_Read__,Gr6_8_Keep_Pace_Math_,Gr6_8_Keep_Pace_Read__,Gr_8_Explore_Math__,Gr_8_Explore_Read__,ISAT_Exceeding_Math__,ISAT_Exceeding_Reading__,ISAT_Value_Add_Math,ISAT_Value_Add_Read,ISAT_Value_Add_Color_Math,ISAT_Value_Add_Color_Read,Students_Taking__Algebra__,Students_Passing__Algebra__,9th Grade EXPLORE (2009),9th Grade EXPLORE (2010),10th Grade PLAN (2009),10th Grade PLAN (2010),Net_Change_EXPLORE_and_PLAN,11th Grade Average ACT (2011),Net_Change_PLAN_and_ACT,College_Eligibility__,Graduation_Rate__,College_Enrollment_Rate__,college_enrollment,General_Services_Route,Freshman_on_Track_Rate__,x_coordinate,y_coordinate,Latitude,Longitude,community_area_number,community_area_name,Ward,Police_District,Location
610038,Abraham Lincoln Elementary School,ES,615 W Kemper Pl,Chicago,IL,60614,(773) 534-5720,http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610038.pdf,Fullerton Elementary Network,NORTH-NORTHWEST SIDE COLLABORATIVE,No,Standard,Not on Probation,Level 1,Yes,Very Strong,99,Very Strong,99,Strong,74,Strong,66,Strong,65,Strong,70,Strong,56,Average,47,96.00%,2.0,96.40%,95.80%,80.1,43.3,89.6,84.9,60.7,62.6,81.9,85.2,52,62.4,66.3,77.9,69.7,64.4,0.2,0.9,Yellow,Green,67.1,54.5,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,813,33,NDA,1171699.458,1915829.428,41.92449696,-87.64452163,7,LINCOLN PARK,43,18,"(41.92449696, -87.64452163)"


In [53]:
%sql select Name_of_School, Average_Student_Attendance from SCHOOLS \
    order by Average_Student_Attendance desc nulls last  limit 10 

 * ibm_db_sa://mxz21962:***@dashdb-txn-sbox-yp-dal09-14.services.dal.bluemix.net:50000/BLUDB
Done.


name_of_school,average_student_attendance
John Charles Haines Elementary School,98.40%
James Ward Elementary School,97.80%
Edgar Allan Poe Elementary Classical School,97.60%
Orozco Fine Arts & Sciences Elementary School,97.60%
Rachel Carson Elementary School,97.60%
Annie Keller Elementary Gifted Magnet School,97.50%
Andrew Jackson Elementary Language Academy,97.40%
Lenart Elementary Regional Gifted Center,97.40%
Disney II Magnet School,97.30%
John H Vanderpoel Elementary Magnet School,97.20%


Double-click __here__ for the solution.

<!-- Solution:

%sql select Name_of_School, Average_Student_Attendance from SCHOOLS \
    order by Average_Student_Attendance desc nulls last limit 10 

-->

### Problem 5

##### Retrieve the list of 5 Schools with the lowest Average Student Attendance sorted in ascending order based on attendance

In [54]:
%sql select Name_of_School, Average_Student_Attendance from SCHOOLS order by Average_Student_Attendance nulls last limit 5

 * ibm_db_sa://mxz21962:***@dashdb-txn-sbox-yp-dal09-14.services.dal.bluemix.net:50000/BLUDB
Done.


name_of_school,average_student_attendance
Richard T Crane Technical Preparatory High School,57.90%
Barbara Vick Early Childhood & Family Center,60.90%
Dyett High School,62.50%
Wendell Phillips Academy High School,63.00%
Orr Academy High School,66.30%


In [55]:
%sql SELECT Name_of_School, Average_Student_Attendance  \
     from SCHOOLS \
     order by Average_Student_Attendance \
     fetch first 5 rows only

 * ibm_db_sa://mxz21962:***@dashdb-txn-sbox-yp-dal09-14.services.dal.bluemix.net:50000/BLUDB
Done.


name_of_school,average_student_attendance
Richard T Crane Technical Preparatory High School,57.90%
Barbara Vick Early Childhood & Family Center,60.90%
Dyett High School,62.50%
Wendell Phillips Academy High School,63.00%
Orr Academy High School,66.30%


Double-click __here__ for the solution.

<!-- Solution:

%sql SELECT Name_of_School, Average_Student_Attendance  \
     from SCHOOLS \
     order by Average_Student_Attendance \
     fetch first 5 rows only

-->


### Problem 6

##### Now remove the '%' sign from the above result set for Average Student Attendance column

Double-click __here__ for a hint

<!--
Use the REPLACE() function to replace '%' with ''
See documentation for this function at:
https://www.ibm.com/support/knowledgecenter/en/SSEPGG_10.5.0/com.ibm.db2.luw.sql.ref.doc/doc/r0000843.html
-->

Double-click __here__ for the solution.

<!-- Hint:

%sql SELECT Name_of_School, REPLACE(Average_Student_Attendance, '%', '') \
     from SCHOOLS \
     order by Average_Student_Attendance \
     fetch first 5 rows only

-->


### Problem 7

##### Which Schools have Average Student Attendance lower than 70%?

Double-click __here__ for a hint

<!--
The datatype of the "Average_Student_Attendance" column is varchar.
So you cannot use it as is in the where clause for a numeric comparison.
First use the CAST() function to cast it as a DECIMAL or DOUBLE
e.g. CAST("Column_Name" as DOUBLE)
or simply: DECIMAL("Column_Name")
-->

Double-click __here__ for another hint

<!--
Don't forget the '%' age sign needs to be removed before casting
-->

Double-click __here__ for the solution.

<!-- Solution:

%sql SELECT Name_of_School, Average_Student_Attendance  \
     from SCHOOLS \
     where CAST ( REPLACE(Average_Student_Attendance, '%', '') AS DOUBLE ) < 70 \
     order by Average_Student_Attendance
     
or,

%sql SELECT Name_of_School, Average_Student_Attendance  \
     from SCHOOLS \
     where DECIMAL ( REPLACE(Average_Student_Attendance, '%', '') ) < 70 \
     order by Average_Student_Attendance

-->


### Problem 8

##### Get the total College Enrollment for each Community Area

Double-click __here__ for a hint

<!--
Verify the exact name of the Enrollment column in the database
Use the SUM() function to add up the Enrollments for each Community Area
-->

Double-click __here__ for another hint

<!--
Don't forget to group by the Community Area
-->

Double-click __here__ for the solution.

<!-- Solution:

%sql select Community_Area_Name, sum(College_Enrollment) AS TOTAL_ENROLLMENT \
   from SCHOOLS \
   group by Community_Area_Name 

-->


### Problem 9

##### Get the 5 Community Areas with the least total College Enrollment  sorted in ascending order 

Double-click __here__ for a hint

<!--
Order the previous query and limit the number of rows you fetch
-->

Double-click __here__ for the solution.

<!-- Solution:

%sql select Community_Area_Name, sum(College_Enrollment) AS TOTAL_ENROLLMENT \
   from SCHOOLS \
   group by Community_Area_Name \
   order by TOTAL_ENROLLMENT asc \
   fetch first 5 rows only

-->

### Problem 10

##### Get the hardship index for the community area which has College Enrollment of 4638

Double-click __here__ for the solution.

<!-- Solution:
NOTE: For this solution to work the CHICAGO_SOCIOECONOMIC_DATA table 
      as created in the last lab of Week 3 should already exist

%%sql 
select hardship_index 
   from chicago_socioeconomic_data CD, schools CPS 
   where CD.ca = CPS.community_area_number 
      and college_enrollment = 4368

-->

### Problem 11

##### Get the hardship index for the community area which has the highest value for College Enrollment

Double-click __here__ for the solution.

<!-- Solution:
NOTE: For this solution to work the CHICAGO_SOCIOECONOMIC_DATA table 
      as created in the last lab of Week 3 should already exist

%sql select ca, community_area_name, hardship_index from chicago_socioeconomic_data \
   where ca in \
   ( select community_area_number from schools order by college_enrollment desc limit 1 )

-->

## Summary

##### In this lab you learned how to work with a real word dataset using SQL and Python. You learned how to query columns with spaces or special characters in their names and with mixed case names. You also used built in database functions and practiced how to sort, limit, and order result sets, as well as used sub-queries and worked with multiple tables.

Copyright &copy; 2018 [cognitiveclass.ai](cognitiveclass.ai?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).
